In [119]:
import pandas as pd
import glob
import numpy as np
import itertools as it

In [120]:
# Read in all of the individual CSV files of the member votes per Congressional session
df = pd.concat([pd.read_csv(f) for f in glob.glob('S*.csv')], ignore_index = True)
df = df[['congress','rollnumber','icpsr','cast_code']]

In [121]:
df.head()

congress  rollnumber  icpsr  cast_code
0        76           1     22          1
1        76           1    193          3
2        76           1    273          1
3        76           1    296          1
4        76           1    349          1

In [144]:
# Read in other files to use later for making data more human readable
cast_codes = pd.read_csv('C:/Users/Laura/Documents/Documents/SMU/Datasets/SenatePolarization/cast_codes.csv')
party_codes = pd.read_csv('C:/Users/Laura/Documents/Documents/SMU/Datasets/SenatePolarization/party_codes.csv')
members = pd.read_csv('C:/Users/Laura/Documents/Documents/SMU/Datasets/SenatePolarization/members.csv')

In [145]:
# Reduce the members data to only the Senate and only those since the 76th congress, to match the voting data
members = members[(members['chamber']=="Senate") & (members['congress']>=76)]
members = members[['congress','icpsr','state_abbrev','party_code','occupancy','bioname']]

In [ ]:
cast_codes

In [124]:
# Create conditional column to summarize Ayes, Nays and Abstentions
conditions = [(df['cast_code']<4) & (df['cast_code']>0), (df['cast_code']>=4) & (df['cast_code']<7)]
choices = [1,-1] # 1 is Aye, -1 is Nay
df['vote'] = np.select(conditions, choices, default = 0)

In [147]:
for j in range(76,115) :
    test1 = df[df['congress']==j]
    for i in range(min(test1['rollnumber']),max(test1['rollnumber'])) :
        intra = test1[(test1['rollnumber']==i)]
        ayes = intra[intra['vote']==1]
        nays = intra[intra['vote']==1]

        ayes1 = pd.DataFrame(list(it.combinations(ayes['icpsr'],2)), columns = ['Sen1','Sen2'])
        ayes1['votecount'] = 1
        nays1 = pd.DataFrame(list(it.combinations(nays['icpsr'],2)), columns = ['Sen1','Sen2'])
        nays1['votecount'] = 1 
        if i == min(test1['rollnumber']) :
            voted_together = pd.concat([ayes1, nays1], ignore_index = True)
        else :
            voted_together = voted_together.append([ayes1,nays1])
    # voted_together.describe()
    
    grouped = voted_together.groupby(['Sen1','Sen2'], as_index=False).sum()
    grouped['congress'] = j
    
    if j == 76 :
        votes = grouped
    else : 
        votes = votes.append(grouped)
votes.head()

Sen1  Sen2  votecount  congress
0    22   193        130        76
1    22   273        150        76
2    22   296        148        76
3    22   349        122        76
4    22   394          2        76

In [148]:
votes.describe()

Sen1           Sen2      votecount       congress
count  204522.000000  204522.000000  204522.000000  204522.000000
mean    10273.896510   19146.133247     440.307449      95.038744
std      9175.146273   19247.420182     284.627794      11.267351
min         9.000000      52.000000       2.000000      76.000000
25%      3388.000000    8270.000000     182.000000      85.000000
50%      9369.000000   14031.000000     422.000000      95.000000
75%     14440.000000   15505.000000     642.000000     105.000000
max     99369.000000   99911.000000    1558.000000     114.000000

In [156]:
data = votes.merge(members, left_on=['Sen1','congress'], right_on = ['icpsr','congress'], suffixes = ['_v','_s1'], how='left')
data = data.merge(members, left_on=['Sen2','congress'], right_on = ['icpsr','congress'], suffixes = ['_s1','_s2'], how='left')
data = data.merge(party_codes, left_on = 'party_code_s1', right_on = 'party_code', suffixes = ['_v','_s1'], how='left')
data = data.merge(party_codes, left_on = 'party_code_s2', right_on = 'party_code', suffixes = ['_s1','_s2'], how='left')
data

Sen1   Sen2  votecount  congress  icpsr_s1 state_abbrev_s1  \
0          22    193        130        76        22              CO   
1          22    273        150        76        22              CO   
2          22    296        148        76        22              CO   
3          22    349        122        76        22              CO   
4          22    394          2        76        22              CO   
5          22    404        136        76        22              CO   
6          22    425        162        76        22              CO   
7          22    437        136        76        22              CO   
8          22    710        120        76        22              CO   
9          22    843        132        76        22              CO   
10         22    858         66        76        22              CO   
11         22   1027        108        76        22              CO   
12         22   1071          2        76        22              CO   
13         22   1131        156        76        22              CO   
14         22   1234        196        76        22              CO   
15         22   1263        134        76        22              CO   
16         22   1365        170        76        22              CO   
17         22   1370        162        76        22              CO   
18         22   1494        192        76        22              CO   
19         22   1497        108        76        22              CO   
20         22   1634         84        76        22              CO   
21         22   1678        170        76        22              CO   
22         22   1760        144        76        22              CO   
23         22   1772        136        76        22              CO   
24         22   1986        142        76        22              CO   
25         22   2317        164        76        22              CO   
26         22   2391        168        76        22              CO   
27         22   2652        110        76        22              CO   
28         22   2702        128        76        22              CO   
29         22   2901        142        76        22              CO   
...       ...    ...        ...       ...       ...             ...   
204492  41503  49706        532       114     41503              NE   
204493  41503  94659        520       114     41503              NE   
204494  41504  41505        664       114     41504              NC   
204495  41504  49300        410       114     41504              NC   
204496  41504  49308        390       114     41504              NC   
204497  41504  49700        566       114     41504              NC   
204498  41504  49703        604       114     41504              NC   
204499  41504  49706        622       114     41504              NC   
204500  41504  94659        562       114     41504              NC   
204501  41505  49300        420       114     41505              SD   
204502  41505  49308        398       114     41505              SD   
204503  41505  49700        578       114     41505              SD   
204504  41505  49703        622       114     41505              SD   
204505  41505  49706        634       114     41505              SD   
204506  41505  94659        568       114     41505              SD   
204507  49300  49308        612       114     49300              CA   
204508  49300  49700        306       114     49300              CA   
204509  49300  49703        512       114     49300              CA   
204510  49300  49706        380       114     49300              CA   
204511  49300  94659        300       114     49300              CA   
204512  49308  49700        290       114     49308              WA   
204513  49308  49703        498       114     49308              WA   
204514  49308  49706        364       114     49308              WA   
204515  49308  94659        290       114     49308              WA   
204516  49700  49703        520  

In [159]:
data1 = data[['congress','bioname_s1','state_abbrev_s1','party_name_s1','bioname_s2','state_abbrev_s2','party_name_s2','votecount']]
data1

congress                                 bioname_s1 state_abbrev_s1  \
0             76                      ADAMS, Alva Blanchard              CO   
1             76                      ADAMS, Alva Blanchard              CO   
2             76                      ADAMS, Alva Blanchard              CO   
3             76                      ADAMS, Alva Blanchard              CO   
4             76                      ADAMS, Alva Blanchard              CO   
5             76                      ADAMS, Alva Blanchard              CO   
6             76                      ADAMS, Alva Blanchard              CO   
7             76                      ADAMS, Alva Blanchard              CO   
8             76                      ADAMS, Alva Blanchard              CO   
9             76                      ADAMS, Alva Blanchard              CO   
10            76                      ADAMS, Alva Blanchard              CO   
11            76                      ADAMS, Alva Blanchard              CO   
12            76                      ADAMS, Alva Blanchard              CO   
13            76                      ADAMS, Alva Blanchard              CO   
14            76                      ADAMS, Alva Blanchard              CO   
15            76                      ADAMS, Alva Blanchard              CO   
16            76                      ADAMS, Alva Blanchard              CO   
17            76                      ADAMS, Alva Blanchard              CO   
18            76                      ADAMS, Alva Blanchard              CO   
19            76                      ADAMS, Alva Blanchard              CO   
20            76                      ADAMS, Alva Blanchard              CO   
21            76                      ADAMS, Alva Blanchard              CO   
22            76                      ADAMS, Alva Blanchard              CO   
23            76                      ADAMS, Alva Blanchard              CO   
24            76                      ADAMS, Alva Blanchard              CO   
25            76                      ADAMS, Alva Blanchard              CO   
26            76                      ADAMS, Alva Blanchard              CO   
27            76                      ADAMS, Alva Blanchard              CO   
28            76                      ADAMS, Alva Blanchard              CO   
29            76                      ADAMS, Alva Blanchard              CO   
...          ...                                        ...             ...   
204492       114                       SASSE, Benjamin Eric              NE   
204493       114                       SASSE, Benjamin Eric              NE   
204494       114               TILLIS, Thomas Roland (Thom)              NC   
204495       114               TILLIS, Thomas Roland (Thom)              NC   
204496       114               TILLIS, Thomas Roland (Thom)              NC   
204497       114               TILLIS, Thomas Roland (Thom)              NC   
204498       114               TILLIS, Thomas Roland (Thom)              NC   
204499       114               TILLIS, Thomas Roland (Thom)              NC   
204500       114               TILLIS, Thomas Roland (Thom)              NC   
204501       114              ROUNDS, Marion Michael (Mike)              SD   
204502       114              ROUNDS, Marion Michael (Mike)              SD   
204503       114              ROUNDS, Marion Michael (Mike)              SD   
204504       114              ROUNDS, Marion Michael (Mike)              SD   
204505       114              ROUNDS, Marion Michael (Mike)              SD   
204506       114              ROUNDS, Marion Michael (Mike)              SD   
204507       114                          FEINSTEIN, Dianne              CA   
204508       114                          FEINSTEIN, Dianne              CA   
204509       114                          FEINSTEIN, Dianne              CA   
204510       114                          FEINSTEIN, Diann

In [160]:
data1.rename({'congress':'congress_session', 
              'bioname_s1':'sen1_name', 'state_abbrev_s1':'sen1_state', 'party_name_s1':'sen1_party', 
              'bioname_s2':'sen2_name', 'state_abbrev_s2':'sen2_state', 'party_name_s2':'sen2_party', 
              'votecount':'votes_together'}, axis='columns')

congress_session                                  sen1_name  \
0                     76                      ADAMS, Alva Blanchard   
1                     76                      ADAMS, Alva Blanchard   
2                     76                      ADAMS, Alva Blanchard   
3                     76                      ADAMS, Alva Blanchard   
4                     76                      ADAMS, Alva Blanchard   
5                     76                      ADAMS, Alva Blanchard   
6                     76                      ADAMS, Alva Blanchard   
7                     76                      ADAMS, Alva Blanchard   
8                     76                      ADAMS, Alva Blanchard   
9                     76                      ADAMS, Alva Blanchard   
10                    76                      ADAMS, Alva Blanchard   
11                    76                      ADAMS, Alva Blanchard   
12                    76                      ADAMS, Alva Blanchard   
13                    76                      ADAMS, Alva Blanchard   
14                    76                      ADAMS, Alva Blanchard   
15                    76                      ADAMS, Alva Blanchard   
16                    76                      ADAMS, Alva Blanchard   
17                    76                      ADAMS, Alva Blanchard   
18                    76                      ADAMS, Alva Blanchard   
19                    76                      ADAMS, Alva Blanchard   
20                    76                      ADAMS, Alva Blanchard   
21                    76                      ADAMS, Alva Blanchard   
22                    76                      ADAMS, Alva Blanchard   
23                    76                      ADAMS, Alva Blanchard   
24                    76                      ADAMS, Alva Blanchard   
25                    76                      ADAMS, Alva Blanchard   
26                    76                      ADAMS, Alva Blanchard   
27                    76                      ADAMS, Alva Blanchard   
28                    76                      ADAMS, Alva Blanchard   
29                    76                      ADAMS, Alva Blanchard   
...                  ...                                        ...   
204492               114                       SASSE, Benjamin Eric   
204493               114                       SASSE, Benjamin Eric   
204494               114               TILLIS, Thomas Roland (Thom)   
204495               114               TILLIS, Thomas Roland (Thom)   
204496               114               TILLIS, Thomas Roland (Thom)   
204497               114               TILLIS, Thomas Roland (Thom)   
204498               114               TILLIS, Thomas Roland (Thom)   
204499               114               TILLIS, Thomas Roland (Thom)   
204500               114               TILLIS, Thomas Roland (Thom)   
204501               114              ROUNDS, Marion Michael (Mike)   
204502               114              ROUNDS, Marion Michael (Mike)   
204503               114              ROUNDS, Marion Michael (Mike)   
204504               114              ROUNDS, Marion Michael (Mike)   
204505               114              ROUNDS, Marion Michael (Mike)   
204506               114              ROUNDS, Marion Michael (Mike)   
204507               114                          FEINSTEIN, Dianne   
204508               114                          FEINSTEIN, Dianne   
204509               114                          FEINSTEIN, Dianne   
204510               114                          FEINSTEIN, Dianne   
204511               114                          FEINSTEIN, Dianne   
204512               114                              MURRAY, Patty   
204513               114                              MURRAY, Patty   
204514               114                              MURRAY, Patty   
204515               114                              MURRAY, Patty   
204516               114  SESSIO

In [161]:
data1.to_csv('voting_history.csv')